In [1]:
import lightgbm as lgb
from sklearn.datasets import load_boston
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score

In [2]:
import os, time, glob, socket,random,csv,pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from timeit import Timer
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.decomposition import PCA 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
pd.set_option("display.max_columns", None)
os.getcwd()
os.chdir(r'D:/fintech/')
path=os.getcwd()
import re

# LightGBM

In [3]:
# only for variable(grade=A)
y_train=pd.read_csv(path+'/grade/A/y_trainA.csv',index_col=0)
y_test=pd.read_csv(path+'/grade/A/y_testA.csv',index_col=0)
X_train_more=pd.read_csv(path+'/grade/A/X_train_more.csv',index_col=0)
X_test_more=pd.read_csv(path+'/grade/A/X_test_more.csv',index_col=0)

X_train_more=X_train_more.drop(columns=['Unnamed: 0.1'])
X_test_more=X_test_more.drop(columns=['Unnamed: 0.1'])



# 處理 y
le = LabelEncoder()
le.fit(y_train['loan_status'].unique())  
y_train['ans']=le.transform(y_train['loan_status'])



In [4]:
X_train_more = X_train_more.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '',x))
X_test_more = X_test_more.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '',x))

In [5]:
# 分 train0 test0

X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2)

# 1

In [7]:
parameters = {
    'max_depth': [10,12,14],
    'device': ['gpu']
}

gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 6,
                         num_leaves = 40,
                         learning_rate = 0.1,
                         feature_fraction = 0.7,
                         min_child_samples=21,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 2,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 0,   
                        )
t0 = time.time()
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='f1', cv=10)
scores = cross_val_score(gbm, X_train0,y_train0.ans, cv=10, scoring='f1')
lgb_A=gsearch.fit(X_train0, y_train0.ans)


print(scores)
t1 = time.time()
print('gpu version elapse time: {}'.format(t1-t0))

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be igno

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be igno

In [9]:
gsearch.best_params_  # 30 70 100 

{'device': 'gpu', 'max_depth': 12}

In [10]:
prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.7378971255673222
0.9712370556374126


In [ ]:
# again

In [15]:
# 分 train0 test0

X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2)


gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 0.7,
                         min_child_samples=21,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 2,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 0,  
                         num_leaves =4096
                        )
lgb_A=gbm.fit(X_train0, y_train0.ans)


prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
0.8664566165777993
0.9885653807043393


# 2

In [16]:
parameters = {
 'boosting' :['gbdt','rf','dart','goss'],
 'device': ['gpu']
}

gbm =  lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 0.7,
                         min_child_samples=21,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 2,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 0,  
                         num_leaves =4096
                        )
t0 = time.time()
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='f1', cv=10)
scores = cross_val_score(gbm, X_train0,y_train0.ans, cv=10, scoring='f1')
lgb_A=gsearch.fit(X_train0, y_train0.ans)


print(scores)
t1 = time.time()
print('gpu version elapse time: {}'.format(t1-t0))

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be igno

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 857, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 617, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py", line 231, in train
    booster = Booster(params=params, train_set=train_set)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\basic.py", line 2061, in __init__
    ctypes.byref(self.handle)))
  File "C:\Users\user\anaconda3\l

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: 

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[0.86220354 0.86630369 0.87246289 0.86586103 0.86319613 0.87314563
 0.87803416 0.85944976 0.86417502 0.86636225]
gpu version elapse time: 1369.7267229557037


In [18]:
gsearch.best_params_  # 30 70 100 

{'boosting': 'gbdt', 'device': 'gpu'}

In [19]:
prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.8676042677012609
0.9886691431298898


In [6]:
# 分 train0 test0

X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2)


gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 0.7,
                         min_child_samples=21,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 2,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 0,  
                         num_leaves =4096
                        )
lgb_A=gbm.fit(X_train0, y_train0.ans)


prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
0.8692094814101793
0.9887210243426651


In [7]:
parameters = {
 'min_child_samples': [18,19,20,21,22],
 'min_child_weight':[0.001,0.002],
 'device': ['gpu']
}

gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 0.7,
                         bagging_fraction = 1,
                         bagging_freq = 2,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 0,
                         num_leaves=4096,
                           
                        )

t0 = time.time()
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='f1', cv=10)
scores = cross_val_score(gbm, X_train0,y_train0.ans, cv=10, scoring='f1')
lgb_A=gsearch.fit(X_train0, y_train0.ans)


print(scores)
t1 = time.time()
print('gpu version elapse time: {}'.format(t1-t0))

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be igno

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be igno

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be igno

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be igno

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be igno

In [9]:
gsearch.best_params_

{'device': 'gpu', 'min_child_samples': 20, 'min_child_weight': 0.001}

In [11]:
prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.8678334339167169
0.9886380144022247


In [14]:
# 分 train0 test0

X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2)


gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 0.7,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 2,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 0,  
                         num_leaves =4096
                        )
lgb_A=gbm.fit(X_train0, y_train0.ans)


prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
0.8660899232362618
0.9885965094320045


# 4

In [15]:
parameters = {
    'feature_fraction': [0.6,0.7,0.8,1],
    'device': ['gpu']
}

gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 0.7,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 2,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 0,
                         num_leaves=4096
                           
                        )


t0 = time.time()
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='f1', cv=10)
scores = cross_val_score(gbm, X_train0,y_train0.ans, cv=10, scoring='f1')
lgb_A=gsearch.fit(X_train0, y_train0.ans)


print(scores)
t1 = time.time()
print('gpu version elapse time: {}'.format(t1-t0))

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be igno

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be igno

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current

In [16]:
gsearch.best_params_

{'device': 'gpu', 'feature_fraction': 1}

In [17]:
prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.8714128709244109
0.9890738165895366


In [20]:
# 分 train0 test0

X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2)


gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 1,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 2,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 0,  
                         num_leaves =4096
                        )
lgb_A=gbm.fit(X_train0, y_train0.ans)


prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
0.8719445457862094
0.9890738165895366


# 5

In [21]:
parameters = {
     'bagging_fraction': [0.8,0.9,1],
     'bagging_freq': [2,3,4],
     'device': ['gpu']
}

gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 1,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 0,
                         num_leaves=4096
                           
                        )


t0 = time.time()
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='f1', cv=10)
scores = cross_val_score(gbm, X_train0,y_train0.ans, cv=10, scoring='f1')
lgb_A=gsearch.fit(X_train0, y_train0.ans)


print(scores)
t1 = time.time()
print('gpu version elapse time: {}'.format(t1-t0))

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] feature_fraction is

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be igno

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be igno

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current

In [22]:
gsearch.best_params_

{'bagging_fraction': 1, 'bagging_freq': 4, 'device': 'gpu'}

In [23]:
a=gsearch.best_params_['bagging_fraction']
b=gsearch.best_params_['bagging_freq']

In [24]:
prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.8687749909123955
0.9887625293128852


In [29]:
# 分 train0 test0

X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2)


gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 1,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         bagging_fraction =1 ,
                         bagging_freq =4 ,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 0,  
                         num_leaves =4096
                        )
lgb_A=gbm.fit(X_train0, y_train0.ans)


prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
0.8673224568138197
0.9885238757341192


# 6

In [30]:
parameters = {
     'cat_smooth': [0,10,20],
     'device': ['gpu']
}

gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 1,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 4,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 0,
                         num_leaves=4096,
                           
                        )

t0 = time.time()
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='f1', cv=10)
scores = cross_val_score(gbm, X_train0,y_train0.ans, cv=10, scoring='f1')
lgb_A=gsearch.fit(X_train0, y_train0.ans)


print(scores)
t1 = time.time()
print('gpu version elapse time: {}'.format(t1-t0))

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

In [31]:
gsearch.best_params_

{'cat_smooth': 10, 'device': 'gpu'}

In [32]:
c=gsearch.best_params_['cat_smooth']

In [33]:
prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.8695235815178358
0.9886898956149999


In [36]:
# 分 train0 test0

X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2)


gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 1,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         bagging_fraction =1 ,
                         bagging_freq =4 ,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 10,  
                         num_leaves =4096
                        )
lgb_A=gbm.fit(X_train0, y_train0.ans)


prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
0.8699390899319239
0.9887002718575549


In [37]:
# save model

with open( path+'\grade\A\datatimeA_LGBM.pickle', 'wb') as f:
    pickle.dump(lgb_A, f)

    
## predict_y

y_pred=lgb_A.predict(X_test_more)
y_ans = pd.DataFrame(y_pred,columns=['predict'],index=y_test.index)
y_output=pd.merge(y_test,y_ans,left_index=True,right_index=True)

y_output.to_csv(path+'\grade\A\y_output_LGBM_datetime_A.csv')

# ADVANCED

# 7

In [7]:
parameters = {
      'reg_alpha': [0.1,0.05,0.005,0.001],
      'reg_lambda': [1,2,3,4,5,6,7,8,9,10],
     'device': ['gpu']
}

gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 1,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 4,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 0,
                         num_leaves=4096,
                           
                        )

t0 = time.time()
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='f1', cv=10)
scores = cross_val_score(gbm, X_train0,y_train0.ans, cv=10, scoring='f1')
lgb_A=gsearch.fit(X_train0, y_train0.ans)


print(scores)
t1 = time.time()
print('gpu version elapse time: {}'.format(t1-t0))


[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 857, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 617, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py", line 252, in train
    booster.update(fobj=fobj)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\basic.py", line 2442, in update
    ctypes.byref(is_finished)))
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\basi

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

...

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

In [9]:
gsearch.best_params_

{'device': 'gpu', 'reg_alpha': 0.1, 'reg_lambda': 3}

In [10]:
d=gsearch.best_params_['reg_alpha']
e=gsearch.best_params_['reg_lambda']

In [15]:
prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.8746250749850031
0.9891568265299769


In [16]:
# 
# 分 train0 test0

X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2)


gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 1,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         bagging_fraction =1 ,
                         bagging_freq =4 ,
                         reg_alpha = 0.1,
                         reg_lambda = 3,
                         cat_smooth = 10,  
                         num_leaves =4096
                        )
lgb_A=gbm.fit(X_train0, y_train0.ans)


prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
0.8726700556765916
0.9890841928320916


# 10--learning-rate

In [6]:
parameters = {
      
     'learning_rate': [0.005,0.001,0.01,0.05,0.1],
     'device': ['gpu']
}

gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 1,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         bagging_fraction =1 ,
                         bagging_freq =4 ,
                         reg_alpha = 0.1,
                         reg_lambda = 3,
                         cat_smooth = 10,  
                         num_leaves =4096
                        )

t0 = time.time()
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='f1', cv=10)
scores = cross_val_score(gbm, X_train0,y_train0.ans, cv=10, scoring='f1')
lgb_A=gsearch.fit(X_train0, y_train0.ans)


print(scores)
t1 = time.time()
print('gpu version elapse time: {}'.format(t1-t0))


[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 857, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 617, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py", line 252, in train
    booster.update(fobj=fobj)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\basic.py", line 2442, in update
    ctypes.byref(is_finished)))
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\basi

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

In [7]:
gsearch.best_params_

{'device': 'gpu', 'learning_rate': 0.01}

In [8]:
prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.8145072658444253
0.9842384875588852


In [10]:
# 分 train0 test0

X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2)


gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate =0.01 ,
                         feature_fraction = 1,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         bagging_fraction =1 ,
                         bagging_freq =4 ,
                         reg_alpha = 0.1,
                         reg_lambda = 3,
                         cat_smooth = 10,  
                         num_leaves =4096
                        )
lgb_A=gbm.fit(X_train0, y_train0.ans)


prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
0.8202537883454478
0.9848610621121879


In [ ]:
# save model

with open( path+'\grade\A\datatimeA_LGBM_1.pickle', 'wb') as f:
    pickle.dump(lgb_A, f)

    
## predict_y

y_pred=lgb_A.predict(X_test_more)
y_ans = pd.DataFrame(y_pred,columns=['predict'],index=y_test.index)
y_output=pd.merge(y_test,y_ans,left_index=True,right_index=True)

y_output.to_csv(path+'\grade\A\y_output_LGBM_datetime_A_1.csv')

In [11]:
parameters = {
      
     'max_bin': [255,300,400,500],
     'device': ['gpu']
}

gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 1,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         bagging_fraction =1 ,
                         bagging_freq =4 ,
                         reg_alpha = 0.1,
                         reg_lambda = 3,
                         cat_smooth = 10,  
                         num_leaves =4096
                        )

t0 = time.time()
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='f1', cv=10)
scores = cross_val_score(gbm, X_train0,y_train0.ans, cv=10, scoring='f1')
lgb_A=gsearch.fit(X_train0, y_train0.ans)


print(scores)
t1 = time.time()
print('gpu version elapse time: {}'.format(t1-t0))


[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 857, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 617, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py", line 252, in train
    booster.update(fobj=fobj)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\basic.py", line 2442, in update
    ctypes.byref(is_finished)))
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\basi

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 857, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 617, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py", line 231, in train
    booster = Booster(params=params, train_set=train_set)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\basic.py", line 2061, in __init__
    ctypes.byref(self.handle)))
  File "C:\Users\user\anaconda3\l

[0.87232743 0.86592866 0.87668712 0.87155963 0.8729963  0.87694656
 0.87268232 0.86841294 0.8794546  0.86185819]
gpu version elapse time: 369.66743087768555


In [12]:
gsearch.best_params_

{'device': 'gpu', 'max_bin': 255}

In [13]:
prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.8764748374668914
0.9893539751385229


In [15]:
# 分 train0 test0

X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2)


gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 12,
                         learning_rate = 0.1,
                         feature_fraction = 1,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         bagging_fraction =1 ,
                         bagging_freq =4 ,
                         reg_alpha = 0.1,
                         reg_lambda = 3,
                         cat_smooth = 10,  
                         num_leaves =4096,
                         max_bin=255
                        )
lgb_A=gbm.fit(X_train0, y_train0.ans)


prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))


[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
0.875945537065053
0.9897897773258347


In [18]:
# save model

with open( path+'\grade\A\datatimeA_LGBM_2.pickle', 'wb') as f:
    pickle.dump(lgb_A, f)

    
## predict_y

y_pred=lgb_A.predict(X_test_more)
y_ans = pd.DataFrame(y_pred,columns=['predict'],index=y_test.index)
y_output=pd.merge(y_test,y_ans,left_index=True,right_index=True)

y_output.to_csv(path+'\grade\A\y_output_LGBM_datetime_A_2.csv')

# final

In [8]:
lgbm_A = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 10,
                         learning_rate = 0.005,
                         feature_fraction = 0.8,
                         min_child_samples=22,
                         min_child_weight=0.001,
                         bagging_fraction =1 ,
                         bagging_freq = 2,
                         reg_alpha = 0.5,
                         reg_lambda = 2,
                         cat_smooth = 20,
                         num_leaves=1024
                           
                        )



t0 = time.time()
#gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='f1', cv=10)
#scores = cross_val_score(gbm, X_train0,y_train0, cv=10, scoring='f1')
lgb_A=lgbm_A.fit(X_train0, y_train0)


#print(scores)
t1 = time.time()
print('gpu version elapse time: {}'.format(t1-t0))





C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
gpu version elapse time: 11.512880802154541


In [ ]:
# save model

with open( path+'\grade\A\datatimeA_LGBM.pickle', 'wb') as f:
    pickle.dump(lgb_A, f)

    
## predict_y

y_pred=lgb_A.predict(X_test_more)
y_ans = pd.DataFrame(y_pred,columns=['predict'],index=y_test.index)
y_output=pd.merge(y_test,y_ans,left_index=True,right_index=True)

y_output.to_csv(path+'\grade\A\y_output_LGBM_datetime_A.csv')

In [10]:
prediction=lgb_A.predict(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.7176007866273353
0.970199431381908
